In [14]:
#!pip install tensorflow
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
import warnings
warnings.filterwarnings("ignore")

## Veri setini indirme

In [15]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

### CIFAR-10 veri seti, sınıf başına 6000 resim olmak üzere 10 sınıfta 60000 adet 32x32 renkli resimden oluşur. 50000 eğitim resmi ve 10000 test resmi vardır.


## Normalizasyon

In [16]:
mean = np.mean(x_train, axis=(0,1,2,3))
std = np.std(x_train, axis=(0,1,2,3))
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

##  One-hot encoding uygulama

In [17]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

## Veri setini ayırma

In [18]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=random_seed)

## Veri artırma

In [19]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

## Model Kurma

### Evrişim + BatchNorm + MaxPooling + Dropout katmanları kullanıldı

In [20]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))

### Adam eniyileme

In [21]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Öğrenme oranı belirleme ve erken durdurma 

In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

## Modeli eğitme ve modelin performansını değerlendirme

In [23]:
history = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                    epochs=50, validation_data=(x_val, y_val),
                    callbacks=[early_stopping, reduce_lr])


test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy:.4f}')

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Tahmin edilen sınıflar
y_true = np.argmax(y_test, axis=1)          # Gerçek sınıflar

f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f'F1 Score: {f1:.4f}')


Epoch 1/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 119s 159ms/step - accuracy: 0.2969 - loss: 2.2051 - val_accuracy: 0.4464 - val_loss: 1.5706 - learning_rate: 0.0010
Epoch 2/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 57s 81ms/step - accuracy: 0.4533 - loss: 1.5023 - val_accuracy: 0.5290 - val_loss: 1.3055 - learning_rate: 0.0010
Epoch 3/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 81s 115ms/step - accuracy: 0.5136 - loss: 1.3534 - val_accuracy: 0.5828 - val_loss: 1.1424 - learning_rate: 0.0010
Epoch 4/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 78s 110ms/step - accuracy: 0.5496 - loss: 1.2620 - val_accuracy: 0.6060 - val_loss: 1.0931 - learning_rate: 0.0010
Epoch 5/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 93s 131ms/step - accuracy: 0.5766 - loss: 1.1873 - val_accuracy: 0.6436 - val_loss: 0.9787 - learning_rate: 0.0010
Epoch 6/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 88s 125ms/step - accuracy: 0.5961 - loss: 1.1450 - val_accuracy: 0.6716 - val_loss: 0.9195 - learning_rate: 0.0010
Epoch 7/50
704/704 ━━━━━━━━━━━━━━━━━━━━ 61s 86ms/step - accuracy: 0.60